Script for scraping old data from dhhs

In [19]:
import pandas as pd
import numpy as np
import calendar
from datetime import date
from datetime import timedelta

In [45]:
pd.options.mode.chained_assignment = None

date formats on the dhhs website:

05-june-2020

9-june-2020

21-june

sunday-12-july

thursday-16-july-2020


In [4]:
def get_active_cases(date):
        
    prefix = 'https://www.dhhs.vic.gov.au/coronavirus-update-victoria-'
        
    y = date.year
    m = calendar.month_name[date.month].lower()
    d = date.day
    wd = calendar.day_name[date.weekday()].lower()
    
    suffixes = []
    
    if (d < 10):
        suffixes.append('0{}-{}-{}'.format(d, m, y))
        suffixes.append('0{}-{}'.format(d, m))
        suffixes.append('{}-0{}-{}'.format(wd, d, m))
        suffixes.append('{}-0{}-{}-{}'.format(wd, d, m, y))

    suffixes.append('{}-{}-{}'.format(d, m, y))
    suffixes.append('{}-{}'.format(d, m))
    suffixes.append('{}-{}-{}'.format(wd, d, m))
    suffixes.append('{}-{}-{}-{}'.format(wd, d, m, y))
        
    df_list = None
    suffix_index = 0
    
    while (df_list == None and suffix_index < len(suffixes)):   
        url = prefix + suffixes[suffix_index]
        try:
            df_list = pd.read_html(url)
        except Exception:
            suffix_index += 1
        
    return df_list[0]

In [171]:
# just leave the rows like 'Total', 'TOTAL', 'Total:', 'UNKNOWN', etc, because 
# there are too many of these that turn up unexpectedly and disappear on later days

# leaving these rows in won't affect the choropleth; they'll just get ignored because they don't
# match any of the feature names in the geojson

def clean_active_cases(df_cases, date):
    df_cases = df_cases[df_cases.iloc[:,0] != 'LGA']
    df_cases.columns = ['lga', 'total', date]
    df_cases.drop('total', axis = 1, inplace = True)
    df_cases.set_index('lga', inplace = True)
    return df_cases

collect data from June 14 to today. put into a single csv.

In [177]:
curr_date = date(2020, 6, 14)
end_date = date.today()

# get the df started with the first date, so all the index names are there.
df_all_dates = get_active_cases(curr_date)
df_all_dates = clean_active_cases(df_all_dates, curr_date)

# now get the cases for the rest of the dates until yesterday
curr_date += timedelta(days=1)

while (curr_date < end_date):
    df_curr = get_active_cases(curr_date)
    df_curr = clean_active_cases(df_curr, curr_date)
    df_all_dates = df_all_dates.merge(df_curr, on = 'lga', how = 'outer')
    curr_date += timedelta(days=1)

# replace all the NaNs with 0, as some of the earlier daily tables used blanks rather than 0s
df_all_dates.replace(np.nan, 0, inplace = True)

In [179]:
# create CSV list of all LGA names
# load it here
# create list of LGA values
# use the list to eliminate all rows where the LGA is not in that list i.e. select df using isin()